In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


from _utils.strategies.trend_simple_filter import (
    apply_buffering_to_position_dict
)
from _utils.strategies.simple_carry import *
from _utils.normalized_price import calculate_normalised_price_dict
from _utils.strategies.trend_asset_classes import (

    calculate_relative_price_dict
    )

from _utils.strategies.breakout import (
    breakout,
    calculate_forecast_for_breakout
)


from _utils.strategies.value import (
    value,
    calculate_forecast_for_value
)

from _utils.strategies.acceleration import (
    accel,
    calculate_forecast_for_accel,
)

from _utils.strategies.skew import (
    skew,
    calculate_forecast_for_skew,
)

from _utils.strategies.dinamic_optmization import *
from _utils.correlation_estimate import *

DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [ ]:
#%% GET DATA
# # =================

def get_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )
    
    carry_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '_carry.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]   
    )

    return adjusted_prices, current_prices, carry_data


def get_fx_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    price = dict(
        [
            (instrument_code, data_for_instrument.price)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )


    return price

In [ ]:
# função auxiliar
def align_dict_to_index(data_dict, target_index):
    for inst in data_dict.keys():
        df = data_dict[inst]

        data_dict[inst] = df.reindex(target_index).ffill().bfill()

    return data_dict




In [ ]:
adjusted_prices_dict, current_prices_dict, carry_prices_dict = get_data_dict(["sp500", "eurostx", "us10", "us2"])

ref = "eurostx"

target_index = adjusted_prices_dict[ref].index
adjusted_prices_dict  = align_dict_to_index(adjusted_prices_dict,  target_index)
current_prices_dict   = align_dict_to_index(current_prices_dict,   target_index)
carry_prices_dict     = align_dict_to_index(carry_prices_dict,     target_index)
fx_data_dict = get_fx_data_dict(['eur_fx'])
multipliers = dict(sp500=5, eurostx=10, us10=1000, us2=2000)
risk_target_tau = 0.2
asset_class_groupings = dict(bonds=["us2", "us10"], stocks=["sp500", "eurostx"])

fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices_dict,dict(sp500=1, eurostx=fx_data_dict['eur_fx'], us10=1, us2=1))

capital = 2000000
idm = 2
instrument_weights = dict(sp500=0.25, eurostx=0.25, us10=0.25, us2=0.25)
cost_per_contract_dict = dict(sp500=0.875, eurostx=6.8, us10=9.5, us2=5.5)


std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=adjusted_prices_dict,
current_prices=current_prices_dict,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

In [ ]:

average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

normalised_price_dict = calculate_normalised_price_dict(
       adjusted_prices_dict=adjusted_prices_dict,
       std_dev_dict=std_dev_dict,
   )


rules_spec_ewmac = [
        dict(function="ewmac", fast_span=16),
        dict(function="ewmac", fast_span=32),
        dict(function="ewmac", fast_span=64),
    ]

list_of_rules_breakout = [
       dict(function=breakout, scalar=0.6, horizon=10),
       dict(function=breakout, scalar=0.67, horizon=20),
       dict(function=breakout, scalar=0.7, horizon=40),
       dict(function=breakout, scalar=0.73, horizon=80),
       dict(function=breakout, scalar=0.74, horizon=160),
       dict(function=breakout, scalar=0.74, horizon=320),
   ]

list_of_rules_value = [dict(function=value, horizon_years=5, scalar=7.27, asset_class_groupings=asset_class_groupings)]


unrounded_position_contracts_dict = calculate_position_dict_with_forecast_applied(
       adjusted_prices_dict=adjusted_prices_dict,
       carry_prices_dict=carry_prices_dict,
       std_dev_dict=std_dev_dict,
       average_position_contracts_dict=average_position_contracts_dict,
       rule_spec=rules_spec_ewmac,
   )


In [ ]:
data_for_optimisation = get_data_for_dynamic_optimisation(
        capital=capital,
        current_prices_dict=current_prices_dict,
        std_dev_dict=std_dev_dict,
        cost_per_contract_dict=cost_per_contract_dict,
        fx_series_dict=fx_series_dict,
        adjusted_prices_dict=adjusted_prices_dict,
        multipliers=multipliers,
        unrounded_position_contracts_dict=unrounded_position_contracts_dict,
    )


In [ ]:
previous_position = get_initial_positions(unrounded_position_contracts_dict)
common_index = data_for_optimisation.common_index
relevant_date = common_index[-1]
algo_to_use = greedy_algo_across_integer_values
data_for_single_period = get_data_for_relevant_date(
            relevant_date, data_for_optimisation=data_for_optimisation
        )


assets_with_data = which_assets_have_data(data_for_single_period)
assets_without_data = which_assets_without_data(
    data_for_single_period, assets_with_data=assets_with_data
)

data_for_single_period = data_for_single_period_with_valid_assets_only(
    data_for_single_period, assets_with_data=assets_with_data
)

previous_position = previous_position.with_selected_assets_only(assets_with_data)

optimised_position = optimisation_for_single_period_with_valid_assets_only(
    previous_position=previous_position,
    data_for_single_period=data_for_single_period,
    algo_to_use=algo_to_use,
)

optimised_position_with_all_assets = (
    optimised_position.with_fill_for_missing_assets(assets_without_data)
)


position_list = []
for relevant_date in common_index:
        data_for_single_period = get_data_for_relevant_date(
            relevant_date, data_for_optimisation=data_for_optimisation
        )

        optimal_positions = optimisation_for_single_period(
            previous_position=previous_position,
            data_for_single_period=data_for_single_period,
            algo_to_use=algo_to_use,
        )

        position_list.append(optimal_positions)
        previous_position = copy(optimal_positions)



optimised_position

{'us2': -2.0, 'eurostx': 0.0, 'us10': 0.0, 'sp500': 0.0}

In [ ]:

average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

normalised_price_dict = calculate_normalised_price_dict(
       adjusted_prices_dict=adjusted_prices_dict,
       std_dev_dict=std_dev_dict,
   )


rules_spec_ewmac = [
        dict(function="ewmac", fast_span=16),
        dict(function="ewmac", fast_span=32),
        dict(function="ewmac", fast_span=64),
    ]

list_of_rules_breakout = [
       dict(function=breakout, scalar=0.6, horizon=10),
       dict(function=breakout, scalar=0.67, horizon=20),
       dict(function=breakout, scalar=0.7, horizon=40),
       dict(function=breakout, scalar=0.73, horizon=80),
       dict(function=breakout, scalar=0.74, horizon=160),
       dict(function=breakout, scalar=0.74, horizon=320),
   ]

list_of_rules_value = [dict(function=value, horizon_years=5, scalar=7.27, asset_class_groupings=asset_class_groupings)]


unrounded_position_contracts_dict = calculate_position_dict_with_forecast_applied(
       adjusted_prices_dict=adjusted_prices_dict,
       carry_prices_dict=carry_prices_dict,
       std_dev_dict=std_dev_dict,
       average_position_contracts_dict=average_position_contracts_dict,
       rule_spec=rules_spec_ewmac,
   )

perc_return_unrounded_dict = calculate_perc_returns_for_dict_with_costs(
       position_contracts_dict=unrounded_position_contracts_dict,
       fx_series=fx_series_dict,
       multipliers=multipliers,
       capital=capital,
       adjusted_prices=adjusted_prices_dict,
       cost_per_contract_dict=cost_per_contract_dict,
       std_dev_dict=std_dev_dict,
   )

# algo_to_use = greedy_algo_across_integer_values

# optimised_positions_dict = dynamically_optimise_positions(
#     capital=capital,
#     current_prices_dict=current_prices_dict,
#     cost_per_contract_dict=cost_per_contract_dict,
#     adjusted_prices_dict=adjusted_prices_dict,
#     fx_series_dict=fx_series_dict,
#     multipliers=multipliers,
#     unrounded_position_contracts_dict=unrounded_position_contracts_dict,
#     std_dev_dict=std_dev_dict,
#     algo_to_use=algo_to_use,
# )

# port_return_rounded_dict = calculate_perc_returns_for_dict_with_costs(
#     position_contracts_dict=optimised_positions_dict,
#     fx_series=fx_series_dict,
#     multipliers=multipliers,
#     capital=capital,
#     adjusted_prices=adjusted_prices_dict,
#     cost_per_contract_dict=cost_per_contract_dict,
#     std_dev_dict=std_dev_dict,
# )

# =================

list_of_rules_accel = [
        dict(function=accel, fast_span=16, scalar=1.87),
        dict(function=accel, fast_span=16, scalar=1.90),
        dict(function=accel, fast_span=32, scalar=1.98),
        dict(function=accel, fast_span=64, scalar=2.05),
    ]

list_of_rules_skew = [
      dict(function=skew, horizon=60, scalar=33.3),
      dict(function=skew, horizon=120, scalar=37.2),
      dict(function=skew, horizon=240, scalar=39.2),
  ]

position_contracts_skew_dict = calculate_position_dict_with_forecast_from_function_applied(
        adjusted_prices_dict=adjusted_prices_dict,
        carry_prices_dict=carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        list_of_rules=list_of_rules_skew,
    )


buffered_position_skew_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_skew_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_skew_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_skew_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)


port_skew_return = aggregate_returns(
    perc_return_skew_dict)

port_skew_return.sort_index(inplace=True)

# ======================================

position_contracts_accel_dict = calculate_position_dict_with_forecast_from_function_applied(
        adjusted_prices_dict=adjusted_prices_dict,
        carry_prices_dict=carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        list_of_rules=list_of_rules_accel,
    )


buffered_position_accel_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_accel_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_accel_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_accel_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)


port_accel_return = aggregate_returns(
    perc_return_accel_dict)

port_accel_return.sort_index(inplace=True)


#==========================

position_contracts_value_dict = calculate_position_dict_with_forecast_from_function_applied(
        adjusted_prices_dict=adjusted_prices_dict,
        carry_prices_dict=carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        list_of_rules=list_of_rules_value,
    )


buffered_position_value_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_value_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_value_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_value_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)


port_value_return = aggregate_returns(
    perc_return_value_dict)

port_value_return.sort_index(inplace=True)

#=====================================

position_contracts_breakout_dict = calculate_position_dict_with_forecast_from_function_applied(
        adjusted_prices_dict=adjusted_prices_dict,
        carry_prices_dict=carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        list_of_rules=list_of_rules_breakout,
    )


buffered_position_breakout_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_breakout_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_breakout_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_breakout_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)


port_breakout_return = aggregate_returns(
    perc_return_breakout_dict)

port_breakout_return.sort_index(inplace=True)

In [ ]:
# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=port_return_rounded_dict.index,
        y=port_return_rounded_dict.cumsum()*100,
        name="Dynamic",
        line=dict(color="orange"),
    )
)

fig.add_trace(
    go.Scatter(
        x=port_skew_return.index,
        y=port_skew_return.cumsum()*100,
        name="Skew",
        line=dict(color="black"),
    )
)

fig.add_trace(
    go.Scatter(
        x=port_accel_return.index,
        y=port_accel_return.cumsum()*100,
        name="Accelaration",
        line=dict(color="green"),
    )
)

fig.add_trace(
    go.Scatter(
        x=port_value_return.index,
        y=port_value_return.cumsum()*100,
        name="Value",
        line=dict(color="blue"),
    )
)


fig.add_trace(
    go.Scatter(
        x=port_breakout_return.index,
        y=port_breakout_return.cumsum()*100,
        name="Breakout",
        line=dict(color="Red"),
    )
)




fig.update_layout(
    title="Strategies for Future Contracts Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=0.8,
        xanchor="left",
        x=0.0),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio Skew Frequência Anual")
print(port_return_rounded_dict, freq=YEAR)
print("================================")
print("Estatísticas Portfolio Skew Frequência Anual")
print(calculate_stats(port_skew_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Acceleration Frequência Anual")
print(calculate_stats(port_accel_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Value Frequência Anual")
print(calculate_stats(port_value_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Breakout trend Frequência Anual")
print(calculate_stats(port_breakout_return, freq=YEAR))
print("================================")
